## Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Environment: Python 3.10.4 (ipykernel)
#### Last Update: 2022-05-27
#### Description: Web crawling code for EURONEXT.com, especially for EURONEXT Brussels. Can be reused for other country by simple editing.

### Utilized BS4 and Selenium Web for Crwaling - Visualized by Pandas

In [5]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version

### Environment
#### - Chromdriver must be located in local PC that is running. This must be modified to be used.

In [6]:
path = r'C:\Users\DT\Desktop\자료\ERUONEXT\\chromedriver'
print(python_version())
print(pd.__version__)

3.10.4
1.4.2


### Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스

In [7]:
cover = open('BelgiumData.csv','w', newline='', encoding='CP949')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### Crawled Website
EURONEXT MAIN: https://live.euronext.com/en /
EURONEXT AMSTERDAM: https://live.euronext.com/en/markets/amsterdam/equities/list

## This is the main code that crawls data rom EURONEXT. 
#### - Time sleep was necessary because of comparably long loading time.
#### - Uncollectable or unavailable data was displayed as "(NA) DataName."
#### - (계산예정) Data Will be calculated and inserted manually later.
#### - Sample Visualization is located in the bottom.

In [9]:
failList = []
#EURONEXT 
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/brussels/equities/list')
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
base_url = "https://live.euronext.com"

#Get each company's link, market symbol, and market code on table - P1
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
all_market_maintable = maintable.find_all(attrs={'td',"stocks-market"})
all_symbol_maintable = maintable.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList3 =[]
for a in all_market_maintable :
    newList3.append(a.text)
newList4 =[]
for b in all_symbol_maintable :
    newList4.append(b.text)

#Get each company's link, market symbol, and market code on table - P2
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[2]').click()
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable2 = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable2 = maintable2.find_all('a')
all_market_maintable2 = maintable2.find_all(attrs={'td',"nowrap pointer"})
all_symbol_maintable2 = maintable2.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList1 =[]
for c in all_market_maintable2 :
    newList1.append(c.text)

newList2 =[]
for d in all_symbol_maintable2 :
    newList2.append(d.text)
    
#Merge two lists (from page 1 and 2)
all_atag_maintable = all_atag_maintable + all_atag_maintable2
all_symbol_maintable = newList3 + newList1
all_market_maintable = newList4 + newList2

#Final Processing to remove unnecessary data
all_symbol_maintable.pop(0)
all_market_maintable.pop(0)
company_links = []
failList = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
#print(len(company_links))

# Main Data Collection code
for sub in company_links:
    driver.get(base_url + sub)
    info = []
    time.sleep(2)
    
    #국가코드
    info.append("BEL")
    info.append("Belgium")
    info.append("Belgie")
    info.append("UTC+02:00")
    info.append("Europe")
    info.append("715000000000 USD")
    info.append("11492641")
    info.append("서유럽")
    
    #기업
    name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
    info.append("BEL XBRU" + name)
    info.append(name)
    info.append(name)
    
    #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
    try :
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(2)
        try:
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros. You can find more information on company's website. Company Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros. You can find more information on company's website. Company Address: " + address)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[1]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
        except:
            info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            ("Belgium")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            info.append("Belgie")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Belgium, Europe")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Belgie, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#cofisem-public-contact-block-empty > div > div.details-wrapper.pt-4.card-body.text-muted > div.field-wrapper.font-weight-medium.mb-1').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#cofisem-public-contact-block-empty > div > div.details-wrapper.pt-4.card-body.text-muted > div.field-wrapper.font-weight-medium.mb-1').text)
        except:
            info.append("")
            info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[1]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[1]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")
    except: 
        ## 소개내용
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Belgium")
        info.append("Belgie")
        info.append("Belgium, Europe")
        info.append("Belgie, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        info.append("")
        info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        
    #재무정보
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(2)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[2]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td').text)
            except:
                info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 유로")
        except:
            info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]').text + "천 유로")      
        except:
            info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
    except : 
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

    #재무정보중 계산예정            
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    #산업군
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
        time.sleep(2)
        try : 
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        except :
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
        try :
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
        except :
            try: 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
    except:
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        
    #언어
    info.append("nl-BE")
    info.append("Nederlands")
    info.append("Dutch")
    
    #주식시장정보
    info.append(str(all_symbol_maintable.pop(0)))
    info.append("EURONEXT BRUSSELS")
    info.append("EURONEXT BRUSSELS")
    info.append(str(all_market_maintable.pop(0)))
    info.append("")
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
    time.sleep(2)
    try:
        info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
    except:
        info.append("")
    info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
    except:
        info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
    except:
        info.append("")

    #지점
    info.append("")
    info.append("")
    info.append("Belgie")
    info.append("Belgie, Europe")
    info.append("")
    info.append("")
    
    #이벤트
    try:
        element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
        href = element.get_attribute('href')
        info.append(href)
    except:
        info.append("")
        
    #화폐
    info.append("EUR")
    info.append("유로")
    
    #관리
    info.append("Chris")
    info.append("EURONEXT")
    info.append("2022-05-26")
    
    #Keep data in csv
    print(info)
    try: 
        writing.writerow(info);
    except:
        failList.append(info);

C:\Users\DT\AppData\Local\Temp\ipykernel_9892\2497416232.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUAB INBEV', 'AB INBEV', 'AB INBEV', 'AB INBEV(English: AB INBEV) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', 'AB INBEV(English: AB INBEV) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', "AB INBEV(English: AB INBEV) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", "AB INBEV(English: AB INBEV) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", '', '', '', '+32 16 276 111', '+32 16 506 111', '', 'https://live.euronext.com/en/product/equities/BE0974293251-XBRU/ab-inbev/abi/quotes', '', '', '', '', '', 'Brouwerijplein 1', 'Brouwerijplein 1', 'BE-3000 Leuven, Vlaams-Brabant', 'BE-3000 Leuven, Vlaams-Brabant', '', '', '', '', 'Active', 'Lauren abbott', 'Lauren abbott', 'CONTACT', 'CONTACT', 'AB INBEV', '+32 16 276 

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUAEDIFICA', 'AEDIFICA', 'AEDIFICA', 'AEDIFICA(English: AEDIFICA) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Rue Belliard 40', 'AEDIFICA(English: AEDIFICA) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Rue Belliard 40', "AEDIFICA(English: AEDIFICA) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 232.118K Euros. You can find more information on company's website. Company Address: Rue Belliard 40", "AEDIFICA(English: AEDIFICA) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 232.118K Euros. You can find more information on company's website. Company Address: Rue Belliard 40", '', '', '', '', '', 'delphine.noirhomme@aedifica.eu', 'https://aedifica.eu/investors/', '', '', '', '', '', 'Rue Belliard 40', 'Rue Belliard 40', 'PO Box 11', 'PO Box 11', '', '', '',

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUARGENX SE', 'ARGENX SE', 'ARGENX SE', 'ARGENX SE(English: ARGENX SE) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Willemstraat 5', 'ARGENX SE(English: ARGENX SE) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Willemstraat 5', "ARGENX SE(English: ARGENX SE) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 439.135K Euros. You can find more information on company's website. Company Address: Willemstraat 5", "ARGENX SE(English: ARGENX SE) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 439.135K Euros. You can find more information on company's website. Company Address: Willemstraat 5", '', '', '', '', '', 'info@argen-x.com', 'https://www.argenx.com/investors', '', '', '', '', '', 'Willemstraat 5', 'Willemstraat 5', 'NL-4811 AH Breda', 'NL-4811 AH Breda', '', 

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUAVANTIUM', 'AVANTIUM', 'AVANTIUM', 'AVANTIUM(English: AVANTIUM) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', 'AVANTIUM(English: AVANTIUM) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', "AVANTIUM(English: AVANTIUM) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", "AVANTIUM(English: AVANTIUM) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", '', '', '', '+31 (0)20 586 80 80', '+31 (0)20 586 80 85', '', 'https://live.euronext.com/en/product/equities/NL0012047823-XAMS/avantium/avtx/quotes', '', '', '', '', '', 'Zekeringstraat 29', 'Zekeringstraat 29', 'NL-1014 BV Amsterdam', 'NL-1014 BV Amsterdam', '', '', '', '', 'Active', 'Caroline van Reedt-Dortland', 'Caroline van Reedt-Dortland', 'CONTACT', 'CONTACT', 'AV

...

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUBONE THERAPEUTICS', 'BONE THERAPEUTICS', 'BONE THERAPEUTICS', 'BONE THERAPEUTICS(English: BONE THERAPEUTICS) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', 'BONE THERAPEUTICS(English: BONE THERAPEUTICS) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', "BONE THERAPEUTICS(English: BONE THERAPEUTICS) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", "BONE THERAPEUTICS(English: BONE THERAPEUTICS) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", '', '', '', '+32 (0)493 09.73.66', '', '', 'https://live.euronext.com/en/product/equities/BE0974280126-XBRU/bone-therapeutics/bothe/quotes', '', '', '', '', '', 'Rue Granbonpré 11', 'Rue Granbonpré 11', 'Bâtiment H (bte 24)', 'Bâtiment H (bte 24)', '', '', '', '', 'Active',

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUCANDELA INVEST', 'CANDELA INVEST', 'CANDELA INVEST', 'CANDELA INVEST(English: CANDELA INVEST) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Route de Lennick 451/32', 'CANDELA INVEST(English: CANDELA INVEST) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Route de Lennick 451/32', "CANDELA INVEST(English: CANDELA INVEST) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 7.168K Euros. You can find more information on company's website. Company Address: Route de Lennick 451/32", "CANDELA INVEST(English: CANDELA INVEST) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 7.168K Euros. You can find more information on company's website. Company Address: Route de Lennick 451/32", '', '', '', '', '', 'cj@vlux.com', 'https://www.candelainvest.com/', '', '', '', '', '', 

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUCHEVRON CORPORATION', 'CHEVRON CORPORATION', 'CHEVRON CORPORATION', 'CHEVRON CORPORATION(English: CHEVRON CORPORATION) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: 6001 Road Bollinger Canyon', 'CHEVRON CORPORATION(English: CHEVRON CORPORATION) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: 6001 Road Bollinger Canyon', "CHEVRON CORPORATION(English: CHEVRON CORPORATION) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 137.412.575K Euros. You can find more information on company's website. Company Address: 6001 Road Bollinger Canyon", "CHEVRON CORPORATION(English: CHEVRON CORPORATION) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 137.412.575K Euros. You can find more information on company's website. Company Address: 6001 Road Bollinger Canyon", '', '', ''

['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUCOFINIMMO', 'COFINIMMO', 'COFINIMMO', 'COFINIMMO(English: COFINIMMO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Boulevard de la Woluwe 58', 'COFINIMMO(English: COFINIMMO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Address: Boulevard de la Woluwe 58', "COFINIMMO(English: COFINIMMO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 299.611K Euros. You can find more information on company's website. Company Address: Boulevard de la Woluwe 58", "COFINIMMO(English: COFINIMMO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). Their net sales were 299.611K Euros. You can find more information on company's website. Company Address: Boulevard de la Woluwe 58", '', '', '', '', '', 'lnachtergaele@cofinimmo.be', 'https://www.cofinimmo.com/investors/', '', '', '', '', '', 'Boulevard de la Woluw

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C0B8F3+2406643]
	Ordinal0 [0x00B9AF31+1945393]
	Ordinal0 [0x00A8C748+837448]
	Ordinal0 [0x00AB92E0+1020640]
	Ordinal0 [0x00AB957B+1021307]
	Ordinal0 [0x00AE6372+1205106]
	Ordinal0 [0x00AD42C4+1131204]
	Ordinal0 [0x00AE4682+1197698]
	Ordinal0 [0x00AD4096+1130646]
	Ordinal0 [0x00AAE636+976438]
	Ordinal0 [0x00AAF546+980294]
	GetHandleVerifier [0x00E79612+2498066]
	GetHandleVerifier [0x00E6C920+2445600]
	GetHandleVerifier [0x00CA4F2A+579370]
	GetHandleVerifier [0x00CA3D36+574774]
	Ordinal0 [0x00BA1C0B+1973259]
	Ordinal0 [0x00BA6688+1992328]
	Ordinal0 [0x00BA6775+1992565]
	Ordinal0 [0x00BAF8D1+2029777]
	BaseThreadInitThunk [0x75C4FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77027A7E+286]
	RtlGetAppContainerNamedObjectPath [0

In [10]:
print(len(failList))
print(failList)

3
[['BEL', 'Belgium', 'Belgie', 'UTC+02:00', 'Europe', '715000000000 USD', '11492641', '서유럽', 'BEL XBRUASCENCIO', 'ASCENCIO', 'ASCENCIO', 'ASCENCIO(English: ASCENCIO) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', 'ASCENCIO(English: ASCENCIO) is a public company that is listed on EURONEXT BRUSSELS(Belgium).', "ASCENCIO(English: ASCENCIO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", "ASCENCIO(English: ASCENCIO) is a public company that is listed on EURONEXT BRUSSELS(Belgium). You can find more information on company's website.", '', '', '', '+32 (0)71 91 95 00', '+32 (0)71 34 48 96', '', 'https://live.euronext.com/en/product/equities/BE0003856730-XBRU/ascencio/asc/quotes', '', '', '', '', '', 'Bâtiment H Avenue Jean Mermoz 1', 'Bâtiment H Avenue Jean Mermoz 1', 'Boîte 4', 'Boîte 4', '', '', '', '', 'Active', 'Aurore Anbergen', 'Aurore Anbergen', 'CONTACT', 'CONTACT', 'ASCENCIO', '+32 (0)71 91